In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from itertools import combinations
import csv

# Example 3D CNN model
class Sample3DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv3d(64, 128, 3, padding=1)
        self.conv5 = nn.Conv3d(128, 256, 3, padding=1)

    def forward(self, x):
        features = []
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        features.append(x)  # fifth-last
        x = F.relu(self.conv3(x))
        features.append(x)  # third-last
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        features.append(x)  # last
        return features

# Initialize model
model = Sample3DCNN()
model.eval()

# Simulated 3D input volumes per region (replace with real data)
regions = {
    "Tibia": torch.randn(1, 1, 32, 32, 32),
    "Femur": torch.randn(1, 1, 32, 32, 32),
    "Background": torch.randn(1, 1, 32, 32, 32)
}

region_features = {region: [] for region in regions}

# Extract pooled feature vectors per region, per layer
with torch.no_grad():
    for region_name, volume in regions.items():
        feature_maps = model(volume)
        pooled_vectors = [F.adaptive_avg_pool3d(fm, 1).flatten(1) for fm in feature_maps]
        region_features[region_name] = pooled_vectors

region_names = list(regions.keys())
layer_names = ['fifth-last', 'third-last', 'last']

# 1. Define the results list with a header row
csv_results = [("Layer", "Region Pair", "Cosine Similarity")]

# 2. Compute and store results
for i, layer in enumerate(layer_names):
    for r1, r2 in combinations(region_names, 2):
        v1 = region_features[r1][i]  # [1, C]
        v2 = region_features[r2][i]  # [1, C]
        sim = F.cosine_similarity(v1, v2).item()
        csv_results.append((layer, f"{r1} ↔ {r2}", sim))

# 3. Write to CSV
with open("cosine_similarities.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(csv_results)

print("Cosine similarities written to 'cosine_similarities.csv'")

Cosine similarities written to 'cosine_similarities.csv'
